In [1]:
#spacy
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

#gensim
import gensim
from gensim import corpora

#Visualization
from spacy import displacy
import pyLDAvis.gensim_models
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import jsonlines

#nltk
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['stopwords','wordnet'])

#warning
import warnings 
warnings.filterwarnings('ignore')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv("Resume.csv")
df = df.reindex(np.random.permutation(df.index))
data = df.copy().iloc[
    0:200,
]
data.head()

,ID,Resume_str,Resume_html,Category
681,93988900,HEALTHCARE CONSULTANT Executive...,"<div class=""fontsize fontface vmargins hmargin...",HEALTHCARE
2147,41590605,UNIT PUBLICIST Summary To ut...,"<div class=""fontsize fontface vmargins hmargin...",PUBLIC-RELATIONS
580,19557384,BUSINESS DEVELOPMENT LEADER Su...,"<div class=""fontsize fontface vmargins hmargin...",BUSINESS-DEVELOPMENT
2172,15856762,"SVP, BUSINESS BANKING Executive...","<div class=""fontsize fontface vmargins hmargin...",BANKING
1525,31948488,FINANCE MANAGER Summary ...,"<div class=""fontsize fontface vmargins hmargin...",FINANCE


In [27]:
nlp = spacy.load(r'C:\Users\DELL\anaconda3\Lib\site-packages\en_core_web_lg\en_core_web_lg-2.3.1')
skill_pattern_path = "jz_skill_patterns.jsonl"

new_ruler = EntityRuler(nlp).from_disk(skill_pattern_path)
nlp.add_pipe(new_ruler)

# ruler = nlp.add_pipe(nlp.create_pipe('entity_ruler'))
# ruler.add_patterns(EntiryRuler(nlp).from_disk(skill_pattern_path))


nlp.pipe_names

# ['tok2vec',
#  'tagger',
#  'parser',
#  'attribute_ruler',
#  'lemmatizer',
#  'ner',
#  'entity_ruler']

['tagger', 'parser', 'ner', 'entity_ruler']

In [9]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [10]:
clean = []
for i in range(data.shape[0]):
    review = re.sub(
        '(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"',
        " ",
        data["Resume_str"].iloc[i],
    )
    review = review.lower()
    review = review.split()
    lm = WordNetLemmatizer()
    review = [
        lm.lemmatize(word)
        for word in review
        if not word in set(stopwords.words("english"))
    ]
    review = " ".join(review)
    clean.append(review)

In [11]:
data["Clean_Resume"] = clean
data["skills"] = data["Clean_Resume"].str.lower().apply(get_skills)
data["skills"] = data["skills"].apply(unique_skills)
data.head()

,ID,Resume_str,Resume_html,Category,Clean_Resume,skills
681,93988900,HEALTHCARE CONSULTANT Executive...,"<div class=""fontsize fontface vmargins hmargin...",HEALTHCARE,healthcare consultant executive profile proven...,"[support, software, brain mapping, certificate..."
2147,41590605,UNIT PUBLICIST Summary To ut...,"<div class=""fontsize fontface vmargins hmargin...",PUBLIC-RELATIONS,unit publicist summary utilize develop skill a...,"[database, twitter, software, search engine, s..."
580,19557384,BUSINESS DEVELOPMENT LEADER Su...,"<div class=""fontsize fontface vmargins hmargin...",BUSINESS-DEVELOPMENT,business development leader summary business d...,"[business, adobe photoshop, finance, material,..."
2172,15856762,"SVP, BUSINESS BANKING Executive...","<div class=""fontsize fontface vmargins hmargin...",BANKING,svp business banking executive profile executi...,"[support, accounting, analytics, mobile, segme..."
1525,31948488,FINANCE MANAGER Summary ...,"<div class=""fontsize fontface vmargins hmargin...",FINANCE,finance manager summary highly organized detai...,"[schedule, accounting, correctness, business a..."


In [ ]:
fig = px.histogram(
    data, x="Category", title="Distribution of Jobs Categories"
).update_xaxes(categoryorder="total descending")
fig.show()